<a href="https://colab.research.google.com/github/urosgodnov/BigData/blob/master/Machine_Learning_with_Python_3_Feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Python to implement machine learning process
by dr. Uros Godnov**

# Feature selection

In [23]:
import pandas as pd
import numpy as np

from google.colab import drive
import sys
# Mount google drive
drive.mount('/content/gdrive')
# Changing path dirctory
sys.path.append('/content/gdrive/MyDrive/Google_Colab_modules')

import sweetviz as sv
import ydata_profiling as ydp

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
df=pd.read_csv("https://raw.githubusercontent.com/urosgodnov/datasets/refs/heads/master/laptop_prices_without_missing_data.csv",sep=";")
df.price_euro=df.price_euro.astype(np.number)
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)


ValueError: could not convert string to float: '4,989E+16'

## Correlation matrix

In [19]:
df.select_dtypes(include=['number','float']).corr()

,inches,cpu_frequency_ghz,ram_gb
inches,1.000000,-0.058888,0.241078
cpu_frequency_ghz,-0.058888,1.000000,-0.018985
ram_gb,0.241078,-0.018985,1.000000


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   company            1275 non-null   object 
 1   product            1275 non-null   object 
 2   typename           1275 non-null   object 
 3   inches             1275 non-null   int64  
 4   screenresolution   1275 non-null   object 
 5   cpu_company        1275 non-null   object 
 6   cpu_type           1275 non-null   object 
 7   cpu_frequency_ghz  1275 non-null   float64
 8   ram_gb             1275 non-null   int64  
 9   memory             1275 non-null   object 
 10  gpu_company        1275 non-null   object 
 11  gpu_type           1275 non-null   object 
 12  opsys              1275 non-null   object 
 13  weight_kg          1275 non-null   object 
 14  price_euro         1275 non-null   object 
dtypes: float64(1), int64(2), object(12)
memory usage: 149.5+ KB
